In [1]:
import pandas as pd
import os
import numpy as np
import requests
import re

In [2]:
df = pd.read_csv("WMT_Grocery_202209_copy.csv")
df = df.drop_duplicates()
df = df.dropna()

/var/folders/9x/pmkb524d571454gzl2dmqb4w0000gn/T/ipykernel_57201/355959686.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("WMT_Grocery_202209_copy.csv")


In [3]:
category = df['CATEGORY'].unique()

In [4]:
def sample_per_category(group):
    return group.sample(n=min(200, len(group)), random_state=42)

In [6]:
filtered_df = df.groupby('CATEGORY', group_keys=False).apply(sample_per_category)
filtered_df.shape
filtered_df.columns

/var/folders/9x/pmkb524d571454gzl2dmqb4w0000gn/T/ipykernel_57201/1767202486.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_df = df.groupby('CATEGORY', group_keys=False).apply(sample_per_category)


Index(['SHIPPING_LOCATION', 'DEPARTMENT', 'CATEGORY', 'BREADCRUMBS', 'SKU',
       'UPC', 'PRODUCT_URL', 'PRODUCT_NAME', 'BRAND', 'PRICE_RETAIL',
       'PRODUCT_SIZE', 'tid', 'AISLE_NUMBER', 'SECTION', 'SHELF_LEVEL',
       'SHELF_POSITION', 'BAY_NUMBER', 'UPC.1'],
      dtype='object')

In [7]:
filtered_df = filtered_df.drop(columns=['SHIPPING_LOCATION','BREADCRUMBS','SKU','UPC.1'])

In [8]:
top_20_categories = [
    'Milk',
    'Cheese',
    'Eggs',
    'Yogurt',
    'Chips',
    'Juices',
    'Chocolate',
    'Frozen Meals & Snacks',
    'Cereal & Granola',
    'Coffee By Type',
    'Tea',
    'Butter & Margarine',
    'Crackers',
    'Snack Nuts',
    'Cookies & Brownies',
    'Salsa & Dips',
    'Energy Drinks',
    'Pasta & pizza',
    'Bacon, Hot Dogs, Sausage',
    'Fresh Fruits'
]


In [9]:
mvp_df = filtered_df[filtered_df['CATEGORY'].isin(top_20_categories)]
mvp_df.shape

(4000, 14)

In [120]:
'''import requests

def generate_hs(desc):
    prompt = f"""You are a strict Health Certifier Bot.

Your job is to assign a health score (0–10) based only on the product name or short description.

Scoring rules:
- 0 = Extremely unhealthy junk food (chips, candy, cookies, soda, ice cream, energy drinks, frozen meals, ultra-processed snacks)
- 1 to 4 = Unhealthy but commonly consumed (flavored popcorn, sugar cereals, snack bars, desserts, frozen foods, sweetened yogurt, processed cheese, white bread)
- 5 to 7 = Moderately healthy or ambiguous (plain pasta, rice, plain bread, canned beans, standard sauces, most packaged foods)
- 8 to 9 = Healthy (plain yogurt, unsalted nuts, minimally processed grains, eggs, lean meats, unflavored tofu)
- 10 = Extremely healthy (fresh produce, fruits, vegetables, plain raw nuts, organic whole foods)

**Instructions:**
- Use only the product name to decide the score.
- If you spot clear junk food keywords (chips, soda, candy, cookies, ice cream, energy drink, bar, dessert, frosted, sweetened, etc.), assign 0–4.
- If the name indicates fresh, organic, whole, or plain foods (carrots, spinach, raw almonds, organic apples), assign 9–10.
- If the name is ambiguous or standard packaged food (pasta, rice, bread, beans, sauce), assign 5–7.
- Do NOT default to 7 unless the name is truly ambiguous.
- Always return only a single number from 0 to 10. No explanation, no extra words.

**Examples:**

Product: "Doritos Nacho Cheese Chips"  
1

Product: "Red Bull Energy Drink Sugar Free"  
2

Product: "Great Value Frozen Chicken Nuggets"  
3

Product: "Barilla Elbows Pasta"  
5

Product: "Kirkland Signature Organic Eggs"  
9

Product: "Fresh Carrots 1 lb"  
10

Product: "Frosted Flakes Cereal"  
2

Product: "Quaker Oats Old Fashioned"  
8

Product: "Yoplait Strawberry Yogurt"  
4

Product: "Plain Greek Yogurt"  
8

Product: "Nature Valley Granola Bar"  
3

Product: "{desc}"
"""

    try:
        response = requests.post(
            'http://localhost:11434/api/generate',
            json={
                "model": "gemma3:1b",
                "prompt": prompt,
                "stream": False
            }
        )
        raw = response.json()['response'].strip()
        score = int(raw)
        print("Score of product is: ", desc, ":", score)
    except Exception as e:
        print("Error:", e)
        score = None

    return score
'''

'import requests\n\ndef generate_hs(desc):\n    prompt = f"""You are a strict Health Certifier Bot.\n\nYour job is to assign a health score (0–10) based only on the product name or short description.\n\nScoring rules:\n- 0 = Extremely unhealthy junk food (chips, candy, cookies, soda, ice cream, energy drinks, frozen meals, ultra-processed snacks)\n- 1 to 4 = Unhealthy but commonly consumed (flavored popcorn, sugar cereals, snack bars, desserts, frozen foods, sweetened yogurt, processed cheese, white bread)\n- 5 to 7 = Moderately healthy or ambiguous (plain pasta, rice, plain bread, canned beans, standard sauces, most packaged foods)\n- 8 to 9 = Healthy (plain yogurt, unsalted nuts, minimally processed grains, eggs, lean meats, unflavored tofu)\n- 10 = Extremely healthy (fresh produce, fruits, vegetables, plain raw nuts, organic whole foods)\n\n**Instructions:**\n- Use only the product name to decide the score.\n- If you spot clear junk food keywords (chips, soda, candy, cookies, ice cr

In [18]:
from transformers import pipeline
import pandas as pd

# 1. Setup your pipeline and labels
classifier = pipeline(
    "zero-shot-classification",
    model="facebook/bart-large-mnli"  # This is much faster than DeBERTa for batch processing!
)

labels = [
    'extremely unhealthy/junk',
    'very unhealthy',
    'unhealthy/processed food',
    'somewhat unhealthy',
    'moderately healthy',
    'neutral',
    'somewhat healthy',
    'healthy',
    'very healthy',
    'extremely healthy/fresh food'
]

labels_to_score = {
    'extremely unhealthy/junk':1,
    'very unhealthy':2,
    'unhealthy/processed food':3,
    'somewhat unhealthy':4,
    'moderately healthy':5,
    'neutral':6,
    'somewhat healthy':7,
    'healthy':8,
    'very healthy':9,
    'extremely healthy/fresh food':10
}

# 2. Get your product names as a list
product_names = mvp_df['PRODUCT_NAME'].tolist()

# 3. Batch process (adjust batch_size as needed for your RAM/CPU)
results = classifier(product_names, labels, batch_size=32, truncation=True)

# 4. Map results to scores
scores = [labels_to_score[result['labels'][0]] for result in results]

# 5. Add the scores as a new column
mvp_df['health_score'] = scores


Device set to use mps:0
/var/folders/9x/pmkb524d571454gzl2dmqb4w0000gn/T/ipykernel_57201/1256892369.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mvp_df['health_score'] = scores


In [19]:
mvp_df

,DEPARTMENT,CATEGORY,UPC,PRODUCT_URL,PRODUCT_NAME,BRAND,PRICE_RETAIL,PRODUCT_SIZE,tid,AISLE_NUMBER,SECTION,SHELF_LEVEL,SHELF_POSITION,BAY_NUMBER,health_score
64113,Deli,"Bacon, Hot Dogs, Sausage",64114,https://www.walmart.com/ip/Oscar-Mayer-Classic...,Oscar Mayer Classic Uncured Wieners Hot Dogs M...,Oscar Mayer,4.88,20,16363349,49,B,3,5,5,3
62923,Deli,"Bacon, Hot Dogs, Sausage",62924,https://www.walmart.com/ip/Cacique-Beef-Choriz...,"Cacique Beef Chorizo, 9 oz Roll",Cacique,1.42,9,16317949,49,B,1,15,5,7
60633,Deli,"Bacon, Hot Dogs, Sausage",60634,https://www.walmart.com/ip/Jack-Link-s-Smokeho...,"Jack Link's Smokehouse, Smoked Sausages, Origi...",Jack Link's,4.98,4,16228207,49,B,3,5,5,5
62965,Deli,"Bacon, Hot Dogs, Sausage",62966,https://www.walmart.com/ip/Armour-Vienna-Sausa...,"Armour Vienna Sausage, Smoked, 4.6 oz Can",Armour,0.86,4.6,16318031,49,B,1,17,7,5
60151,Deli,"Bacon, Hot Dogs, Sausage",60152,https://www.walmart.com/ip/Great-Value-Mild-It...,"Great Value Mild Italian Sausage, 5 Links, 1 l...",Great Value,5.28,3,16216975,49,B,1,3,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119709,Dairy & Eggs,Yogurt,119710,https://www.walmart.com/ip/Chobani-with-Zero-S...,"Chobani with Zero Sugar, Sugar Free Greek Yogu...",Chobani,1.22,5.3,16255374,15,C,4,16,6,8
118863,Dairy & Eggs,Yogurt,118864,https://www.walmart.com/ip/El-Viajero-Bio-Salu...,El Viajero Bio Salud Strawberry Cultured Dairy...,El Viajero,1.12,2.1,16233478,15,C,4,10,10,7
116409,Dairy & Eggs,Yogurt,116410,https://www.walmart.com/ip/Noosa-Yoghurt-Whole...,Noosa Yoghurt Whole Milk Yogurt Lemon 8 oz,noosa,1.98,8,16181207,15,C,4,16,6,7
116182,Dairy & Eggs,Yogurt,116183,https://www.walmart.com/ip/Clio-Strawberry-Gre...,Clio Strawberry Greek Yogurt and Granola Parfa...,Clio,1.98,1.94,16175922,15,C,5,9,9,7


In [20]:
category = mvp_df['CATEGORY']
for i in category:
    mvp_df[mvp_df['CATEGORY']==i].to_csv(f"{i}.csv",index=None)